## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 10
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_10


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 27.5 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Buffalo Bills,9,2022,27.8,50.0,7.9,294.2,95.2,12.3,16.8,...,10.4,109.6,20.000000,11.666667,12.666667,0.094444,2.222222,0.777778,1.285714,2.000000
1,Kansas City Chiefs,9,2022,30.0,75.0,8.0,313.9,105.2,12.0,22.9,...,10.4,99.0,20.888889,12.000000,12.777778,0.055556,2.777778,0.555556,3.800000,3.571429


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[9, 10]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_10,0.700133,0.259749,0.163303,0.313972,0.379176,0.363267,0.419295
1,Kansas City Chiefs,2022,week_10,0.385637,0.295492,0.192964,0.319744,0.409874,0.320742,0.406845
3,Philadelphia Eagles,2022,week_10,0.300945,0.265670,0.145042,0.258683,0.362319,0.266532,0.354647


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_10,0.700133,0.259749,0.163303,0.313972,0.379176,0.363267,0.419295
1,Kansas City Chiefs,2022,week_10,0.385637,0.295492,0.192964,0.319744,0.409874,0.320742,0.406845
3,Philadelphia Eagles,2022,week_10,0.300945,0.265670,0.145042,0.258683,0.362319,0.266532,0.354647
2,Miami Dolphins,2022,week_10,0.074864,0.280978,0.120186,0.276535,0.374377,0.225388,0.336937
8,San Francisco 49ers,2022,week_10,0.191982,0.059368,0.068310,0.189954,0.342020,0.170327,0.323265
13,Seattle Seahawks,2022,week_10,0.012243,0.054839,0.058485,0.195959,0.348906,0.134087,0.306824
21,Denver Broncos,2022,week_10,0.061641,0.153300,0.100169,0.199308,0.341488,0.171181,0.306507
19,Dallas Cowboys,2022,week_10,0.109361,0.040860,0.069231,0.201151,0.334407,0.151002,0.306277
7,Cincinnati Bengals,2022,week_10,0.020421,0.129897,0.051104,0.200572,0.346860,0.149771,0.306055
15,Tampa Bay Buccaneers,2022,week_10,0.029480,0.072168,0.071544,0.205928,0.344500,0.144724,0.305122
